In [1]:
%matplotlib widget
from termcolor import colored
import kagglehub
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from algorithms.utils import LinearRegressionModel
# Download latest version
path = kagglehub.dataset_download("nikhil7280/student-performance-multiple-linear-regression")
files = os.listdir(path)
files = os.path.join(path, files[0])
print(files)

C:\Users\DELL\.cache\kagglehub\datasets\nikhil7280\student-performance-multiple-linear-regression\versions\1\Student_Performance.csv


In [7]:
from sklearn.datasets import load_breast_cancer as dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
df = dataset(as_frame=True).frame
X, y = dataset(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [4]:
X.shape

(569, 30)

In [5]:
y.shape

(569,)

In [6]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [7]:
df.target.unique()

array([0, 1])

# 1 From scratch

In [12]:
x_mean = x_train.mean(axis=0)
x_std = x_train.std(axis=0)

x_norm = (x_train - x_mean) / x_std
############ 

x_test_norm = (x_test - x_mean) / x_std

In [13]:
def sigmoid(z):
    '''
    Args:
    z (ndarray): shape(m,) the linear comination

    Returns: 
    s (ndarray): shape(m,), sigmoid of linear combination
    '''
    s = 1 / (1 + np.exp(-z))
    return s

In [14]:
def predict(x, w, b, threshold=0.5):
    z = x @ w + b
    probs = sigmoid(z)               # probabilities between 0 and 1
    labels = (probs >= threshold).astype(int)  # threshold default 0.5
    return labels, probs


In [34]:
def cost(x, y, w, b, _lambda=1, regu=False):
    '''
    Args:
    x (ndarray): shape (m, n). m examples with n features
    y (ndarray): shape (m,). true labels (0 or 1)
    w (ndarray): shape (n,). model parameters
    b (scalar): model parameter (bias)

    Returns:
    c (scalar): the cost of the model
    '''
    m = x.shape[0]
    z = x @ w + b
    f_wb = sigmoid(z)
    f_wb = np.clip(f_wb, 1e-10, 1 - 1e-10)

    loss = y * np.log(f_wb) + (1 - y) * np.log(1 - f_wb)
    c = -np.mean(loss)

    if regu:
        c += (_lambda / (2 * m)) * np.sum(np.square(w))  # scale properly by number of examples

    return c

In [16]:
def gradient_calc(x, y, w, b):

    m = x.shape[0]
    z = x @ w + b
    f_wb = sigmoid(z)
    error = f_wb - y
    dj_db = np.mean(error)

    dj_dw = (x.T @ error) / m

    return dj_dw, dj_db

In [17]:
def gradient_dec(x, y, w, b, alpha=0.01, iters=1000, show_process=True, _lambda=1, regu=False):
    """
    Gradient descent for logistic regression.

    Args:
        x (ndarray): shape (m, n) feature matrix
        y (ndarray): shape (m,) target labels
        w (ndarray): shape (n,) initial weights
        b (float): initial bias
        alpha (float): learning rate
        iters (int): number of iterations
        show_process (bool): whether to print progress

    Returns:
        w (ndarray): learned weights
        b (float): learned bias
        cost_history (list): cost per iteration
    """
    print_count = max(1, iters // 10)
    cost_history = []
    m = x.shape[0]
    for i in range(iters):
        dj_dw, dj_db = gradient_calc(x, y, w, b)   # gradient
        if regu:
            dj_dw += (_lambda / m) * w

        # update weights and bias
        w = w - alpha * dj_dw
        b = b - alpha * dj_db

        # compute cost
        c = cost(x, y, w, b, _lambda=_lambda, regu=regu)
        cost_history.append(c)

        # optional printing
        if i % print_count == 0 and show_process:
            print(f"Iteration {i}: {colored(f'cost = {c:.4f}', 'red')}, w = {w}, b = {b:.4f}")

    return w, b, cost_history


In [18]:
w = np.zeros((30,))
b = 0
alpha = 12
iters = 10000
_lambda=1
w, b, _ = gradient_dec(x_norm, y_train, w, b, alpha, iters, _lambda=_lambda, regu=True)

Iteration 0: cost = 1.0973, w = [-4.2507041  -2.37655181 -4.32563724 -4.14147542 -2.27679613 -3.54535757
 -4.01749602 -4.52751117 -2.08040947 -0.00596454 -3.28036384  0.13884376
 -3.20391345 -3.29269088  0.40930626 -1.69069855 -1.4674671  -2.4298247
 -0.07410485 -0.34375846 -4.50709498 -2.61678099 -4.54765212 -4.28403094
 -2.49174716 -3.49664976 -3.88000596 -4.63356383 -2.49868416 -1.90008685], b = 1.5165
Iteration 1000: cost = 0.0655, w = [-0.505009   -0.55875709 -0.47136777 -0.53876338 -0.20776881  0.6537183
 -0.60657357 -0.70002515 -0.17303646  0.1936224  -1.08420605  0.2511597
 -0.53722113 -0.93492873 -0.1581156   0.64274526  0.17124982 -0.44709437
  0.35874651  0.4355783  -0.94511339 -1.24834093 -0.76147279 -0.95384106
 -0.75297406  0.04382188 -0.8288045  -0.94845311 -0.93516727 -0.17209461], b = 0.3005
Iteration 2000: cost = 0.0655, w = [-0.505009   -0.55875709 -0.47136777 -0.53876338 -0.20776881  0.6537183
 -0.60657357 -0.70002515 -0.17303646  0.1936224  -1.08420605  0.2511597
 

In [19]:
w_ori = w / x_std
b_ori = b - np.sum((w * x_mean) / x_std)
print(w_ori, b_ori)

[-1.44472710e-01 -1.26977345e-01 -1.95404407e-02 -1.56360316e-03
 -1.54556342e+01  1.22275323e+01 -7.42830494e+00 -1.80039495e+01
 -6.17027779e+00  2.70741984e+01 -4.05264279e+00  4.39940741e-01
 -2.72520597e-01 -2.24110675e-02 -5.04854148e+01  3.48438863e+01
  5.29616809e+00 -7.11616105e+01  4.09796954e+01  1.56848374e+02
 -1.98354262e-01 -2.00742276e-01 -2.29645954e-02 -1.72697502e-03
 -3.34277537e+01  2.74390648e-01 -3.84240981e+00 -1.42174452e+01
 -1.44867246e+01 -9.35933531e+00] 34.49301462126563


In [20]:
print('train cost:', cost(x_train, y_train, w_ori, b_ori))
print('test cost:', cost(x_test, y_test, w_ori, b_ori))

train cost: 0.05135379953587599
test cost: 0.07787201038618774


In [33]:
y_pred_train, _ = predict(x_norm, w, b)
y_pred_test, _ = predict(x_test_norm, w, b)

train_acc = np.mean(y_pred_train == y_train)
test_acc = np.mean(y_pred_test == y_test)
print(f"Train accuracy: {train_acc:.4f}")
print(f"Test accuracy: {test_acc:.4f}")

Train accuracy: 0.9890
Test accuracy: 0.9825


In [22]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_test)
print("Confusion matrix (test set):")
print(cm)

Confusion matrix (test set):
[[41  1]
 [ 1 71]]


In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Precision: 0.9861
Recall: 0.9861
F1-score: 0.9861


# 2 Using sklearn

In [25]:
from sklearn.linear_model import LogisticRegression

In [11]:
# Scaling setse
scaler = StandardScaler()

In [24]:
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [27]:
lr = LogisticRegression()

In [28]:
lr.fit(x_train_scaled, y_train)

LogisticRegression()

In [31]:
lr.score(x_test_scaled, y_test)

0.9824561403508771